# Imports

In [22]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, IntegerType
from pyspark.sql.functions import col, lit, sum, avg, max, min, mean, count, udf, when
from pyspark import SparkConf, SparkContext

In [2]:
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

In [4]:
df_office_project = spark.read.csv("data/office_project.csv", header=True, inferSchema=True)
df_office_project.show(4)

+-----------+-----------------+----------+-----+------+---+-----+
|employee_id|    employee_name|department|state|salary|age|bonus|
+-----------+-----------------+----------+-----+------+---+-----+
|       1000|        Nitz Leif| Marketing|   CA|  6131| 26|  543|
|       1001|  Melissia Dedman|   Finance|   AK|  4027| 43| 1290|
|       1002|Rudolph Barringer|        HR|   LA|  3122| 43| 1445|
|       1003|      Tamra Amber|  Accounts|   AK|  5717| 47| 1291|
+-----------+-----------------+----------+-----+------+---+-----+
only showing top 4 rows



In [5]:
df_office_project.count()

1000

In [6]:
df_office_project.select("department").distinct().show()

+----------+
|department|
+----------+
|     Sales|
|        HR|
|   Finance|
|Purchasing|
| Marketing|
|  Accounts|
+----------+



In [8]:
display(df_office_project.select("department").distinct().count())
df_office_project.select("department").dropDuplicates(["department"]).show()

6

+----------+
|department|
+----------+
|     Sales|
|        HR|
|   Finance|
|Purchasing|
| Marketing|
|  Accounts|
+----------+



In [10]:
df_office_project.groupBy("state").agg(
    count("state").alias("employee_count"), sum("salary").alias("salary_sum")
).sort(col("salary_sum").desc()).show()

+-----+--------------+----------+
|state|employee_count|salary_sum|
+-----+--------------+----------+
|   WA|           208|   1121926|
|   LA|           205|   1112424|
|   CA|           205|   1105013|
|   AK|           209|   1086468|
|   NY|           173|    904113|
+-----+--------------+----------+



In [11]:
df_office_project.groupBy("department").agg(
    count("state").alias("employee_count"), sum("salary").alias("salary_sum")
).sort(col("salary_sum").desc()).show()

+----------+--------------+----------+
|department|employee_count|salary_sum|
+----------+--------------+----------+
|        HR|           171|    988537|
|     Sales|           169|    929430|
| Marketing|           170|    881983|
|Purchasing|           166|    852551|
|  Accounts|           162|    841873|
|   Finance|           162|    835570|
+----------+--------------+----------+



In [13]:
df_office_project.groupBy("state", "department").agg(
    count("state").alias("employee_count"), sum("salary").alias("salary_sum")
).sort(col("state").asc()).show()

+-----+----------+--------------+----------+
|state|department|employee_count|salary_sum|
+-----+----------+--------------+----------+
|   AK| Marketing|            42|    222636|
|   AK|Purchasing|            30|    162663|
|   AK|  Accounts|            37|    175763|
|   AK|        HR|            25|    142949|
|   AK|     Sales|            38|    198180|
|   AK|   Finance|            37|    184277|
|   CA|        HR|            28|    155708|
|   CA| Marketing|            33|    169825|
|   CA|Purchasing|            32|    161203|
|   CA|     Sales|            42|    249432|
|   CA|   Finance|            35|    200392|
|   CA|  Accounts|            35|    168453|
|   LA|     Sales|            35|    199421|
|   LA|Purchasing|            45|    252488|
|   LA|   Finance|            29|    131111|
|   LA|  Accounts|            29|    161007|
|   LA|        HR|            41|    235166|
|   LA| Marketing|            26|    133231|
|   NY|  Accounts|            34|    179785|
|   NY|   

In [15]:
df_office_project.groupBy("department").agg(
    min('salary'), max("salary"), avg("salary")
).sort(col("department").asc()).show()

+----------+-----------+-----------+-----------------+
|department|min(salary)|max(salary)|      avg(salary)|
+----------+-----------+-----------+-----------------+
|  Accounts|       1007|       9890|5196.746913580247|
|   Finance|       1006|       9899| 5157.83950617284|
|        HR|       1013|       9982|5780.918128654971|
| Marketing|       1031|       9974|5188.135294117647|
|Purchasing|       1105|       9985|5135.849397590361|
|     Sales|       1103|       9982|5499.585798816568|
+----------+-----------+-----------+-----------------+



In [18]:
# select states in NY where avg salary is greater than the avg salary of NY
avg_ny_salary = df_office_project.filter(col("state") == "NY") \
    .groupBy("state").agg(avg("salary").alias("avg_salary")) \
    .select('avg_salary').collect()[0]['avg_salary']

display(type(avg_ny_salary))
display(avg_ny_salary)

float

5226.086705202312

In [21]:
# select states in NY where avg salary is greater than the avg salary of NY
# and the department is Finance
df_office_project.filter(
    (col("state") == "NY") & (col('salary') > avg_ny_salary)
    & (col('department') == 'Finance')).show()

+-----------+--------------------+----------+-----+------+---+-----+
|employee_id|       employee_name|department|state|salary|age|bonus|
+-----------+--------------------+----------+-----+------+---+-----+
|       1079|       Grigg Debroah|   Finance|   NY|  5728| 28| 1146|
|       1087|       Leif Lemaster|   Finance|   NY|  8642| 45| 1782|
|       1100|Ellingsworth Meli...|   Finance|   NY|  7845| 32| 1358|
|       1180|       Juliana Grigg|   Finance|   NY|  8178| 42| 1617|
|       1286|        Kohn Vankirk|   Finance|   NY|  5754| 39| 1229|
|       1301|     Melissia Katlyn|   Finance|   NY|  5731| 26|  525|
|       1335|    Deandre Melissia|   Finance|   NY|  8898| 32|  513|
|       1342|   Phylicia Antonina|   Finance|   NY|  8626| 30| 1857|
|       1509|   Ellingsworth Nitz|   Finance|   NY|  7234| 45| 1004|
|       1660|       Melissia Jere|   Finance|   NY|  9201| 21| 1533|
|       1737|      Yukiko Kreamer|   Finance|   NY|  7396| 20| 1332|
|       1738|      Nena Kensinger|

# UDF

In [ ]:
# Raise the salaries $500 of all employees whose age is greater than 45
df_office_project.withColumn("salary",
    when(col("age") > 45, col("salary") + 500).otherwise(col("salary"))
).show()

In [24]:
def inc_salary(age, cur_salary):
    if age > 45:
        return cur_salary + 500
    return cur_salary

udf_inc_salary = udf(inc_salary, IntegerType())

df_office_project.withColumn(
    "salary_inc", udf_inc_salary(col("age"), col("salary"))).show()

+-----------+-------------------+----------+-----+------+---+-----+----------+
|employee_id|      employee_name|department|state|salary|age|bonus|salary_inc|
+-----------+-------------------+----------+-----+------+---+-----+----------+
|       1000|          Nitz Leif| Marketing|   CA|  6131| 26|  543|      6131|
|       1001|    Melissia Dedman|   Finance|   AK|  4027| 43| 1290|      4027|
|       1002|  Rudolph Barringer|        HR|   LA|  3122| 43| 1445|      3122|
|       1003|        Tamra Amber|  Accounts|   AK|  5717| 47| 1291|      6217|
|       1004|        Mullan Nitz|Purchasing|   CA|  5685| 34| 1394|      5685|
|       1005|      Zollner Karie|  Accounts|   CA|  2843| 27| 1078|      2843|
|       1006|Kaczorowski Zollner|     Sales|   CA|  7201| 21| 1834|      7201|
|       1007|      Nakano Locust| Marketing|   LA|  3444| 23| 1823|      3444|
|       1008|  Recalde Kensinger|  Accounts|   LA|  3704| 48| 1330|      4204|
|       1009|        Imai Hallie|  Accounts|   AK|  

# write / file output

In [25]:
# get number of partitions for df_office_project
df_office_project.rdd.getNumPartitions()

1

In [26]:
# export all employees whose age is greater than 45 to a csv file
df_office_project.filter(col("age") > 45).write \
    .csv("data/office_project_gt_45.csv")